## Camada Silver – Tratamento e Padronização

Este notebook aplica regras de limpeza, conversão de tipos e padronização textual aos dados da CEAP, mantendo a granularidade original.



**Objetivo da camada Silver:** 
Transformar os dados brutos da camada Bronze em dados confiáveis, tipados, padronizados e consistentes, mantendo a granularidade original, sem ainda aplicar modelagem analítica.

**Visão geral do mapeamento**

| Ação       | Bronze    | Silver                   |
| ---------- | --------- | ------------------------ |
| Estrutura  | CSV cru   | Tabela Spark estruturada |
| Tipos      | String    | Tipos corretos           |
| Datas      | Texto ISO | Date                     |
| Valores    | String    | Decimal                  |
| Duplicatas | Mantidas  | Avaliadas                |
| Outliers   | Mantidos  | Identificados            |
| Semântica  | Original  | Preservada               |


Na camada Silver foram aplicadas transformações de padronização de tipos, normalização textual, conversão de datas e valores monetários, além da avaliação de valores nulos e inconsistentes. Não foram realizadas agregações ou alterações de granularidade, mantendo-se a correspondência de uma linha por despesa.

**Tabela Silver proposta:**

SILVER_DESPESA_CEAP

####Mapeamento campo a campo:
**Identificadores e metadados**
| Campo Bronze | Tipo Bronze | Campo Silver | Tipo Silver | Regra          |
| ------------ | ----------- | ------------ | ----------- | -------------- |
| idDocumento  | string      | id_documento | string      | trim, não nulo |
| ano          | string/int  | ano          | int         | cast int       |

**Parlamentar**
| Campo Bronze    | Campo Silver     | Tipo   | Regra        |
| --------------- | ---------------- | ------ | ------------ |
| idDeputado      | id_parlamentar   | int    | cast         |
| nomeParlamentar | nome_parlamentar | string | trim + upper |
| siglaPartido    | sigla_partido    | string | trim + upper |
| siglaUF         | sigla_uf         | string | trim + upper |
| legislatura     | legislatura      | int    | cast         |

**Datas**
| Campo Bronze | Campo Silver | Tipo | Regra     |
| ------------ | ------------ | ---- | --------- |
| datEmissao   | data_despesa | date | parse ISO |
| mes          | mes          | int  | cast      |
| ano          | ano          | int  | cast      |

**Valores monetários**
| Campo Bronze | Campo Silver  | Tipo          | Regra |
| ------------ | ------------- | ------------- | ----- |
| vlrDocumento | valor_despesa | decimal(12,2) | cast  |
| vlrGlosa     | valor_glosa   | decimal(12,2) | cast  |
| vlrLiquido   | valor_liquido | decimal(12,2) | cast  |

**Tipo de despesa**
| Campo Bronze   | Campo Silver      | Tipo   | Regra |
| -------------- | ----------------- | ------ | ----- |
| codTipoDespesa | id_tipo_despesa   | int    | cast  |
| txtDescricao   | descricao_despesa | string | trim  |

**Fornecedor**
| Campo Bronze      | Campo Silver        | Tipo   | Regra           |
| ----------------- | ------------------- | ------ | --------------- |
| txtFornecedor     | fornecedor          | string | trim            |
| cnpjCpfFornecedor | cnpj_cpf_fornecedor | string | somente dígitos |
| txtCNPJCPF        | cnpj_cpf_formatado  | string | manter original |

**Localização da despesa**
| Campo Bronze | Campo Silver | Tipo   | Regra        |
| ------------ | ------------ | ------ | ------------ |
| txtMunicipio | municipio    | string | trim         |
| txtUF        | uf_despesa   | string | trim + upper |
| txtPais      | pais         | string | default = BR |

**Documento fiscal**
| Campo Bronze | Campo Silver     | Tipo   | Regra  |
| ------------ | ---------------- | ------ | ------ |
| urlDocumento | url_documento    | string | manter |
| numDocumento | numero_documento | string | trim   |


In [0]:
spark.sql("USE CATALOG mvp_ed_ceap")
spark.sql("USE SCHEMA layer_silver")

DataFrame[]

In [0]:
from pyspark.sql.functions import col, to_date, trim, upper

df_bronze = spark.table("mvp_ed_ceap.layer_bronze.bronze_ceap_despesas")

df_silver = (
    df_bronze
    .withColumn("id_documento", trim(col("ideDocumento")))
    .withColumn("id_parlamentar", col("idDeputado").cast("int"))
    .withColumn("nome_parlamentar", upper(trim(col("nomeParlamentar"))))
    .withColumn("sigla_partido", upper(trim(col("siglaPartido"))))
    .withColumn("sigla_uf", upper(trim(col("siglaUF"))))
    .withColumn("data_despesa", to_date(col("datEmissao")))
    .withColumn("valor_despesa", col("vlrDocumento").cast("decimal(12,2)"))
    .withColumn("id_tipo_despesa", col("codTipoDespesa").cast("int"))
)

(
    df_silver
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("silver_ceap_despesas")
)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4728930721504161>, line 22
      3 df_bronze = spark.table("mvp_ed_ceap.layer_bronze.bronze_ceap_despesas")
      5 df_silver = (
      6     df_bronze
      7     .withColumn("id_documento", trim(col("idDocumento")))
   (...)
     14     .withColumn("id_tipo_despesa", col("codTipoDespesa").cast("int"))
     15 )
     17 (
     18     df_silver
     19     .write
     20     .mode("overwrite")
     21     .format("delta")
---> 22     .saveAsTable("silver_ceap_despesas")
     23 )

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.co